### CCKS系统分为以下几部分：  
1. 后端。分为两个项目，ccks2项目实现与mysql交互，数据库中保存病历、标注结果等数据；  
neo4jtest项目实现与Neo4j图数据库的交互（主要原因是不同版本的spring boot对于图数据库的语法支持不一样）
2. 前端。实现数据统计、病历数据库检索、实体标注、知识图谱等功能  
3. 算法端。实现模型预测，将结果批量保存到mysql和neo4j，目前未实现自动化

### 1.后端

#### ccks

**如何启动**
1. 通过mysql运行sql文件夹中的02-ccks.sql，得到数据库
2. 修改application.yml中的数据库账户密码，以及url: jdbc:mysql://localhost:3306/你的数据库名?useUnicode
3. 运行Ccks2Application

**具体介绍**  
  
![markdown](./img/02-ccks2-java%E6%96%87%E4%BB%B6%E5%A4%B9%E7%9B%AE%E5%BD%95.PNG)  
先来看java文件夹下的目录  
+ Ccks2Application：用于启动项目
+ config：用代码的形式配置项目  
    + CORSConfig：关闭前端页面和后端不同端口之间的跨域保护（请自行搜索CORS）
    + MbgConfig：mybatis-generator的配置，可独立运行，根据数据库已有的表直接生成常用增删改查代码
    + MybatisConfig：mybatis的配置，配合上一条使用，通过注解@MapperScan指定Mapper的位置
+ mapper：自动生成，用于与数据库交互的接口
    + XXXMapper：都是自动生成，实际sql语句在**resource/com/yang/ccks2/mapper/XXXMapper.xml**
    + CcksTextDao：对于极为简单的sql，可直接手写，注意有注解@Mapper，功能相当于MapperScan
+ service：业务逻辑
    + XXXService：自动生成，都是抽象接口，通过这种方式封装方法，实现其他程序调用时的即插即用
    + iml：
        + XXXServiceImpl 业务逻辑的具体实现，部分方法通过PageHelper实现分页查询
+ controller：实现与前端访问请求的交互
    + XXXController：只负责实现通用逻辑，如对访问请求的处理，获得URL中携带的参数以及请求中携带的json数据，具体逻辑都在service
+ model：自动生成，数据库对应的类
    + XXXEntity：与数据表一一对应
    + XXXEntityExample：用于XXXMapper的辅助类，实现代码风格增删改查（where关键词）
+ Repository：一个失败的探索，没什么用
+ utils：同上

resource文件夹中：  
+ application.yml：项目配置信息，需要注意的是：  
  ``` java
  mybatis:
    mapper-locations:
      - classpath:mapper/*.xml
      - classpath*:com/**/mapper/*.xml
  ```
  代表具体sql语句的位置，详细理论请搜索mybatis-generator  
+ generator.properties
+ generatorConfig.xml：都是mybatis-generator的配置文件
+ db：一些数据库的更改记录，没用


#### neo4jtest

结构没什么特殊的，重点在于XXXRepository，理解其中的CQL语句即可（请先学习neo4j相关知识）

如何启动：  
创建neo4j数据库，设置数据库名和密码  
参考https://www.csdn.net/tags/MtTaYg5sMjI3MjQtYmxvZwO0O0OO0O0O.html 将数据导入neo4j  
具体数据和CQL语句在sql\desktop-csv-import  
启动数据库
更改application.yml中neo4j的对应名称和密码  
启动Neo4jtestApplication